In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#reading dataset
df_train = pd.read_csv("/kaggle/input/ga-customer-revenue-prediction/train.csv")


#reading dataset
df_test = pd.read_csv("/kaggle/input/ga-customer-revenue-prediction/test.csv")




In [ ]:
#convert json columns

import os
import json
from pandas.io.json import json_normalize

def load_df(csv_path='/kaggle/input/ga-customer-revenue-prediction/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
#load the train and test dataset

train_df = load_df('/kaggle/input/ga-customer-revenue-prediction/train.csv')


test_df = load_df('/kaggle/input/ga-customer-revenue-prediction/test.csv')

In [ ]:
#imputation of null values and converting the columns values to int in train dataset

def fill_na(df):   
    df['totals.pageviews'].fillna(1, inplace=True)
    df['totals.newVisits'].fillna(0, inplace=True)
    df['totals.bounces'].fillna(0, inplace=True) 
    df["totals.transactionRevenue"].fillna(0.0, inplace=True)
    
    # Changing datatypes from object to desired ones
    df['totals.pageviews'] = df['totals.pageviews'].astype(int)
    df['totals.newVisits'] = df['totals.newVisits'].astype(int)
    df['totals.bounces'] = df['totals.bounces'].astype(int)
    df["totals.transactionRevenue"] = df["totals.transactionRevenue"].astype(float)
    
    
    df['trafficSource.isTrueDirect'].fillna(False, inplace=True) 
    df['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True, inplace=True) # filling boolean with True
    df[train_df['geoNetwork.city'] == "(not set)"]['geoNetwork.city'] = np.nan
    df['geoNetwork.city'].fillna("NaN", inplace=True)
    
    return df

df = fill_na(train_df)

In [ ]:
#imputation of null values and converting the columns values to int in test data

def fill_na(df1):   
    df1['totals.pageviews'].fillna(1, inplace=True)
    df1['totals.newVisits'].fillna(0, inplace=True)
    df1['totals.bounces'].fillna(0, inplace=True)
    
    # Changing datatypes from object to desired ones
    df1['totals.pageviews'] = df1['totals.pageviews'].astype(int)
    df1['totals.newVisits'] = df1['totals.newVisits'].astype(int)
    df1['totals.bounces'] = df1['totals.bounces'].astype(int)
    
    
    df1['trafficSource.isTrueDirect'].fillna(False, inplace=True) 
    df1['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True, inplace=True) # filling boolean with True
    df1[train_df['geoNetwork.city'] == "(not set)"]['geoNetwork.city'] = np.nan
    df1['geoNetwork.city'].fillna("NaN", inplace=True)
    
    return df1

df1 = fill_na(test_df)

In [ ]:
#converting to float fullVisitorId & sessionId to float

df['fullVisitorId'] = df['fullVisitorId'].astype(float)

df['sessionId'] = df['sessionId'].astype(float)



df1['fullVisitorId'] = df1['fullVisitorId'].astype(float)

df1['sessionId'] = df1['sessionId'].astype(float)

In [ ]:
#Removing columns with unique values in train

for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)

In [ ]:

#Removing columns with unique values in train

for col in df1.columns:
    if len(df1[col].unique()) == 1:
        df1.drop(col,inplace=True,axis=1)

In [ ]:
# This function is to extract date features in train

from datetime import datetime


def date_process(df):
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d") # seting the column as pandas datetime
    df["weekday"] = df['date'].dt.weekday #extracting week day
    df["day"] = df['date'].dt.day # extracting day
    df["month"] = df['date'].dt.month # extracting day
    df["year"] = df['date'].dt.year # extracting day
    df['visitHour'] = (df['visitStartTime'].apply(lambda x: str(datetime.fromtimestamp(x).hour))).astype(int)
    
    return df
df = date_process(df)


In [ ]:
# This function is to extract date features in test

from datetime import datetime


def df1_date(df1):
    df1["date"] = pd.to_datetime(df1["date"], format="%Y%m%d") # seting the column as pandas datetime
    df1["weekday"] = df1['date'].dt.weekday #extracting week day
    df1["day"] = df1['date'].dt.day # extracting day
    df1["month"] = df1['date'].dt.month # extracting day
    df1["year"] = df1['date'].dt.year # extracting day
    df1['visitHour'] = (df1['visitStartTime'].apply(lambda x: str(datetime.fromtimestamp(x).hour))).astype(int)
    
    return df1
df1 = df1_date(df1)


In [ ]:
#Drop session id and trafficSource.campaignCode as they give no value to revenuw

df = df.drop(['sessionId', 'trafficSource.campaignCode'], axis = 1)
df1 = df1.drop(['sessionId'], axis = 1)


In [ ]:
#fill totals.transactionRevenue columns with 0 for nan data and convert it to values.
#convert full visitor id to values

df["totals.transactionRevenue"].fillna(0, inplace=True)
train_y = df["totals.transactionRevenue"].values
train_id = df["fullVisitorId"].values
test_id = df1["fullVisitorId"].values


In [ ]:
from sklearn import model_selection, preprocessing, metrics

# function to label encode the categorical variables 

df_cat = ["channelGrouping", "device.browser", 'device.isMobile',
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']

In [ ]:
#label encoding the categorical variable in train and test

for col in df_cat:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(df[col].values.astype('str')) + list(df1[col].values.astype('str')))
    df[col] = lbl.transform(list(df[col].values.astype('str')))
    df1[col] = lbl.transform(list(df1[col].values.astype('str')))


In [ ]:
#convert the these columns to float

num_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    
for col in num_cols:
    df[col] = df[col].astype(float)
    df1[col] = df1[col].astype(float)


In [ ]:
import datetime

#converting date to date time

df["date"] = pd.to_datetime(df["date"]).dt.date



In [ ]:
X = df[df['date']<=datetime.date(2017,5,31)]

val_X = df[df['date']>datetime.date(2017,5,31)]
X = X.drop(['date'], axis = 1)
val_X = val_X.drop(['date'], axis = 1)
test_X = df1.drop(['date'], axis = 1)

In [ ]:
y = np.log1p((X["totals.transactionRevenue"]).values)
val_y = np.log1p((val_X["totals.transactionRevenue"]).values)


In [ ]:
x = X.drop(['totals.transactionRevenue'], axis = 1)
val_x = val_X.drop(['totals.transactionRevenue'], axis = 1)


In [ ]:
y = pd.DataFrame(y)
val_y = pd.DataFrame(val_y)

In [ ]:
#drop columns that doesnt affects revenue

x1 = x.drop(['day', 'totals.bounces', 'fullVisitorId', 'visitId', 'visitStartTime', 'trafficSource.adwordsClickInfo.adNetworkType',
          'trafficSource.adwordsClickInfo.isVideoAd' ], axis = 1)
val_x1 = val_x.drop(['day', 'totals.bounces', 'fullVisitorId', 'visitId', 'visitStartTime', 'trafficSource.adwordsClickInfo.adNetworkType',
          'trafficSource.adwordsClickInfo.isVideoAd' ], axis = 1)
test_x = test_X.drop(['weekday', 'totals.bounces', 'fullVisitorId', 'visitId', 'visitStartTime', 'trafficSource.adwordsClickInfo.adNetworkType',
          'trafficSource.adwordsClickInfo.isVideoAd' ], axis = 1)


In [ ]:
import xgboost as xg 
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score


model = xg.XGBRegressor(objective ='reg:squarederror',n_estimators = 1000, verbosity=1, learning_rate=0.3, max_depth=8,
                       subsample=0.5, colsample_bytree=0.5, scale_pos_weight=3)
model.fit(x1, y)

In [ ]:
model.score(x1,y)

In [ ]:
y_train_predict = model.predict(x1)
rmse = (np.sqrt(MSE(y, y_train_predict)))
r2 = r2_score(y, y_train_predict)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

# model evaluation for testing set

y_test_predict = model.predict(val_x1)
# root mean square error of the model
rmse = (np.sqrt(MSE(val_y, y_test_predict)))

# r-squared score of the model
r2 = r2_score(val_y, y_test_predict)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

In [ ]:
#pickle file with xgboost model
import pickle

model_file = "model.sav"
with open(model_file,mode='wb') as model_f:
    pickle.dump(model,model_f)


In [ ]:

with open(model_file,mode='rb') as model_f:
    model = pickle.load(model_f)
    result = model.score(val_x1,val_y)
    print("result:",result)


In [ ]:
Pkl_Filename = "Pickle_XG_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)

# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    Pickled_XG_Model = pickle.load(file)

Pickled_XG_Model


LGB Model starts here...

In [ ]:
#Model with lgbm

from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb


def run_lgb(x1, y, val_x1, val_y, test_x):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 30,
        "min_child_samples" : 100,
        "learning_rate" : 0.05,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : 1,
        "max_depth" : 10
        
    }
    
    lgtrain = lgb.Dataset(x1, label=y)
    lgval = lgb.Dataset(val_x1, label=val_y)
    model_lgbm = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100)
    
    pred_test_y = model_lgbm.predict(test_x, num_iteration=model_lgbm.best_iteration)
    pred_val_y = model_lgbm.predict(val_x1, num_iteration=model_lgbm.best_iteration)
    return pred_test_y, model_lgbm, pred_val_y

# Training the model #
pred_test, model_lgbm, pred_val = run_lgb(x1, y, val_x1, val_y, test_x)

In [ ]:
y_train_predict = model_lgbm.predict(x1)
rmse = (np.sqrt(MSE(y, y_train_predict)))
r2 = r2_score(y, y_train_predict)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

In [ ]:
y_train_predict = model_lgbm.predict(x1)
rmse = (np.sqrt(MSE(y, y_train_predict)))
r2 = r2_score(y, y_train_predict)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

# model evaluation for testing set

y_test_predict = model_lgbm.predict(val_x1)
# root mean square error of the model
rmse = (np.sqrt(MSE(val_y, y_test_predict)))

# r-squared score of the model
r2 = r2_score(val_y, y_test_predict)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

In [ ]:
Pkl_Filename = "lgb_model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model_lgbm, file)

# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    lgb_model = pickle.load(file)

lgb_model
